# Analysis of aircraft incidents




## Teil 3
### 3.1 Teilen Sie Ihren Datensatz in einen test- und einen train-Satz ein.

Zuerst teile ich den Datensatz in *features und targets* auf.
features = x (Aircraft_Model, Incident_Cause, Aircraft_Phase etc.)
targest = y (Aircaft_Damage_Type)

Dann Teilen wir den Datensatz mit einer 10% zu 90%. Ich denke das über 2000 Zeilen genügen um mein Modell zu Validieren.
Ausserdem bemerkte ich das Aircaft_Model ein Tippfehler im Datensatz hat (sollte Airc**r**aft sein).

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

path = r"C:\Users\RinSk\Downloads\Aircraft_Incident_Dataset.csv"
df = pd.read_csv(path, sep=";", on_bad_lines="skip")

# Eingabedaten und Zielspalte definieren
X = df[["Aircaft_Model", "Incident_Cause(es)", "Aircraft_Phase", "Onboard_Total", "Aircaft_Engines"]]
y = df["Fatalities"]

# In Trainings- und Testdaten aufteilen (10% Testdaten)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

print("Trainingsdaten:", len(X_train))
print("Testdaten:", len(X_test))

Trainingsdaten: 21167
Testdaten: 2352


### 3.2 Bestimmen Sie einen Algorithmus aus sklearn, der für Ihren Datensatz und Ihr Problem geeignet ist.

| Algorithmus                 | Beschreibung                                                                                                    |
| --------------------------- | --------------------------------------------------------------------------------------------------------------- |
| **Linear Regression**       | Findet eine lineare Beziehung zwischen Eingabedaten und Zielwerten. Gut bei einfachen, linearen Zusammenhängen. |
| **Decision Tree Regressor** | Lernt Entscheidungsregeln aus den Daten. Gut bei nichtlinearen und kategorischen Daten.                         |
| **Random Forest Regressor** | Ein Ensemble aus vielen Entscheidungsbäumen. Sehr leistungsfähig, robust gegenüber Ausreißern und Overfitting.  |

Da mein Datensatz gemischte Daten enthält (Zahlen und Text), ist es nicht klar, ob der Zusammenhang linear ist. Deshalb teste ich mehrere Modelle und vergleichen sie mit MSE (Fehlermaß) und R² (Erklärte Varianz).


In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression  # Much faster than RandomForest
from sklearn.metrics import mean_squared_error, r2_score

# Load data
path = r"C:\Users\RinSk\Downloads\Aircraft_Incident_Dataset.csv"
df = pd.read_csv(path, sep=";", on_bad_lines="skip")

# Simplify data preparation - focus only on key predictors
# Extract numeric info from strings for numerical columns
df['Engine_Count'] = df['Aircaft_Engines'].str.extract(r'(\d+)').astype(float)
df['Engine_Count'] = df['Engine_Count'].fillna(2)  # Default to 2 engines if missing

# Keep only essential columns that could predict fatalities
categorical_columns = ["Aircraft_Phase"]  # Keep only the most relevant categorical column
numerical_columns = ["Engine_Count"]     # Use our extracted numeric engine count

# Create feature dataframe and target
X = df[categorical_columns + numerical_columns]
y = df["Fatalities"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# Simpler preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        ('num', 'passthrough', numerical_columns)
    ],
    sparse_threshold=0  # Force dense output
)

# Use a simpler, faster model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())  # Much faster than RandomForest
])

# Train model
print("Training model...")
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Training successful!")
print(f"MSE: {mse:.2f}")
print(f"R²-Wert: {r2:.2f}")

Training model...
Training successful!
MSE: 225.70
R²-Wert: 0.12


Es sieht so aus als würde nur 12% der Predictions stimmen. Daran liegen könnte es an dem ausgewählten Algorithmus sein oder an dem sehr **unkonsistenten Datensatz** der hauptsächlich aus **strings** besteht.wie zum Beispiel die Reihe die ich versuchte vorherzusagen hat so eine Struktur:

Fatalities: 0 / Occupants: 7
Fatalities: 0 / Occupants: 23

Ausserdem gibt es **keine Balance** nur wenige haben mehrere Fatalities. Meistens (glücklicherweise) gab es 0 weshalb dies stark das training (und das Resultat) beinflusste.

### 3.3 Testen
Ich nehme Manuell Test daten und vergleiche sie.

In [26]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import random

# Get a small sample of test data with predictions
sample_size = min(5, len(X_test))
sample_indices = random.sample(range(len(X_test)), sample_size)

# Get the sample data
sample_X = X_test.iloc[sample_indices]
sample_y_true = y_test.iloc[sample_indices]
sample_y_pred = model.predict(sample_X)

# Create a DataFrame to display the results
results = pd.DataFrame({
    'Phase': sample_X['Aircraft_Phase'],
    'Engine_Count': sample_X['Engine_Count'],
    'Actual_Fatalities': sample_y_true,
    'Predicted_Fatalities': np.round(sample_y_pred, 1)
})

# Display the results
print("Sample predictions for evaluation:")
print(results)

# Calculate overall metrics
mae = mean_absolute_error(y_test, y_pred)
print(f"\nMean Absolute Error: {mae:.2f}")
print(f"R²-Wert: {r2:.2f}")

Sample predictions for evaluation:
                Phase  Engine_Count  Actual_Fatalities  Predicted_Fatalities
19761  En route (ENR)           2.0                 14                  10.4
16887   Takeoff (TOF)           2.0                  0                   2.0
11117   Landing (LDG)           2.0                  0                   0.1
8025    Unknown (UNK)           2.0                  0                   1.2
13346  Approach (APR)           4.0                  6                  22.7

Mean Absolute Error: 6.52
R²-Wert: 0.12


Zusammenfassung der Modellevaluierung:

Das Modell zur Vorhersage von Flugunfallopfern zeigt eine mäßige Leistung mit einem R²-Wert von nur 0,12. Bei der manuellen Überprüfung von Testdaten fällt auf, dass das Modell bei Unfällen mit keinen Todesopfern relativ genaue Vorhersagen macht, jedoch bei schweren Unfällen die Opferzahlen oft unterschätzt. Dies liegt vermutlich an der unausgewogenen Datenverteilung, da die meisten Vorfälle keine Todesopfer haben. Das Modell hat Schwierigkeiten, die komplexen Umstände zu erfassen, die zu hohen Todeszahlen führen. Verbesserungspotenzial besteht in der Einbeziehung zusätzlicher Faktoren wie Flugzeugtyp, Besatzungsgröße und detailliertere Unfallursachen.